# Qubit operations

In [1]:
using Revise
using Plots
using LinearAlgebra
using MultiScales

newaxis = [CartesianIndex()]

1-element Vector{CartesianIndex{0}}:
 CartesianIndex()

In [2]:
using ITensors

println(ITensors.blas_get_num_threads())

1


In [3]:
function eval(psi, sites, el)
    V = ITensor(1.)
    for j=1:length(el)
        V *= (psi[j] * state(sites[j], el[j]))
    end
    return scalar(V)
end

eval (generic function with 2 methods)

In [4]:
nbit = 2
N = 2^nbit

sites = siteinds("Qubit", nbit)

2-element Vector{Index{Int64}}:
 (dim=2|id=700|"Qubit,Site,n=1")
 (dim=2|id=276|"Qubit,Site,n=2")

In [5]:
Z = [op("Z", sites, n) for n in 1:nbit]
;

In [6]:
mps = MPS(sites, ["1" for n in 1:nbit])

MPS
[1] ((dim=2|id=700|"Qubit,Site,n=1"), (dim=1|id=973|"Link,l=1"))
[2] ((dim=1|id=973|"Link,l=1"), (dim=2|id=276|"Qubit,Site,n=2"))


In [7]:
M = MPO(sites, ["Z" for n in 1:nbit])

MPO
[1] ((dim=2|id=700|"Qubit,Site,n=1")', (dim=2|id=700|"Qubit,Site,n=1"), (dim=1|id=112|"Link,l=1"))
[2] ((dim=2|id=276|"Qubit,Site,n=2")', (dim=2|id=276|"Qubit,Site,n=2"), (dim=1|id=112|"Link,l=1"))


In [8]:
# Create Z_1 X_2
# See: https://github.com/ITensor/ITensors.jl/blob/main/src/mps/mpo.jl#L69
os = Prod{Op}()
os *= Op("Z", 1)
os *= Op("X", 2)
M = MPO(os, sites)

MPO
[1] ((dim=3|id=135|"Link,l=1"), (dim=2|id=700|"Qubit,Site,n=1")', (dim=2|id=700|"Qubit,Site,n=1"))
[2] ((dim=3|id=135|"Link,l=1"), (dim=2|id=276|"Qubit,Site,n=2")', (dim=2|id=276|"Qubit,Site,n=2"))


In [9]:
# Create Z_1
M = MPO(Op("Z", 1), sites)

MPO
[1] ((dim=3|id=433|"Link,l=1"), (dim=2|id=700|"Qubit,Site,n=1")', (dim=2|id=700|"Qubit,Site,n=1"))
[2] ((dim=3|id=433|"Link,l=1"), (dim=2|id=276|"Qubit,Site,n=2")', (dim=2|id=276|"Qubit,Site,n=2"))


In [10]:
# Control: 1
# Target: 2
cnot = op("CNOT", sites, 1, 2)

state0 = MPS(sites, ["1" for n in 1:nbit])
for i2 in 1:2, i1 in 1:2
    @show i1, i2, eval(state0, sites, [i1, i2])
end
res = noprime(contract(MPO(cnot, sites), state0))

(i1, i2, eval(state0, sites, [i1, i2])) = (1, 1, 0.0)
(i1, i2, eval(state0, sites, [i1, i2])) = (2, 1, 0.0)
(i1, i2, eval(state0, sites, [i1, i2])) = (1, 2, 0.0)
(i1, i2, eval(state0, sites, [i1, i2])) = (2, 2, 1.0)


MPS
[1] ((dim=2|id=896|"Link,l=1"), (dim=2|id=700|"Qubit,Site,n=1"))
[2] ((dim=2|id=276|"Qubit,Site,n=2"), (dim=2|id=896|"Link,l=1"))


In [11]:
for i2 in 1:2, i1 in 1:2
    @show i1, i2, eval(res, sites, [i1, i2])
end

(i1, i2, eval(res, sites, [i1, i2])) = (1, 1, 0.0)
(i1, i2, eval(res, sites, [i1, i2])) = (2, 1, 1.0)
(i1, i2, eval(res, sites, [i1, i2])) = (1, 2, 0.0)
(i1, i2, eval(res, sites, [i1, i2])) = (2, 2, 0.0)


In [12]:
# Phase π: 1
# Phase π/2: 2
os = Prod{Op}()
os *= Op("Phase", 1; ϕ=π)
#os *= Op("Phase", 2; ϕ=π/2)
M = MPO(os, sites)
#M = MPO(Prod{Op}(Op("Phase", 1; ϕ=π)), sites)

state0 = MPS(sites, ["1" for n in 1:nbit])
for i2 in 1:2, i1 in 1:2
    @show i1, i2, eval(state0, sites, [i1, i2])
end
res = noprime(contract(M, state0))

#for i2 in 1:2, i1 in 1:2
    #@show i1, i2, eval(res, sites, [i1, i2])
#end

(i1, i2, eval(state0, sites, [i1, i2])) = (1, 1, 0.0)
(i1, i2, eval(state0, sites, [i1, i2])) = (2, 1, 0.0)
(i1, i2, eval(state0, sites, [i1, i2])) = (1, 2, 0.0)
(i1, i2, eval(state0, sites, [i1, i2])) = (2, 2, 1.0)


MPS
[1] ((dim=2|id=918|"Link,l=1"), (dim=2|id=700|"Qubit,Site,n=1"))
[2] ((dim=2|id=276|"Qubit,Site,n=2"), (dim=2|id=918|"Link,l=1"))


In [13]:
state0 = MPS(sites, ["1" for n in 1:nbit])
for i2 in 1:2, i1 in 1:2
    @show i1, i2, eval(state0, sites, [i1, i2])
end
println("")
println("")

@show state0[1]
println("")
println("")

state0[1] = noprime(state0[1] * op("Phase", sites[1]; ϕ=π))
;

(i1, i2, eval(state0, sites, [i1, i2])) = (1, 1, 0.0)
(i1, i2, eval(state0, sites, [i1, i2])) = (2, 1, 0.0)
(i1, i2, eval(state0, sites, [i1, i2])) = (1, 2, 0.0)
(i1, i2, eval(state0, sites, [i1, i2])) = (2, 2, 1.0)


state0[1] = 

ITensor ord=2
Dim 1: (dim=2|id=700|"Qubit,Site,n=1")
Dim 2: (dim=1|id=872|"Link,l=1")
NDTensors.Dense{Float64, Vector{Float64}}
 2×1
 0.0
 1.0




In [14]:
for i2 in 1:2, i1 in 1:2
    @show "B ", i1, i2, eval(state0, sites, [i1, i2])
end

("B ", i1, i2, eval(state0, sites, [i1, i2])) = ("B ", 1, 1, 0.0 + 0.0im)
("B ", i1, i2, eval(state0, sites, [i1, i2])) = ("B ", 2, 1, -0.0 + 0.0im)
("B ", i1, i2, eval(state0, sites, [i1, i2])) = ("B ", 1, 2, 0.0 + 0.0im)
("B ", i1, i2, eval(state0, sites, [i1, i2])) = ("B ", 2, 2, -1.0 + 1.2246467991473532e-16im)


In [27]:
sites = siteinds("Qubit", 4)
state0 = MPS(sites, ["1" for n in 1:4])
@show inds(state0[1])

inds(state0[1]) = ((dim=2|id=288|"Qubit,Site,n=1"), (dim=1|id=184|"Link,l=1"))


((dim=2|id=288|"Qubit,Site,n=1"), (dim=1|id=184|"Link,l=1"))

In [26]:
@show inds(state0[2])

inds(state0[2]) = ((dim=1|id=12|"Link,l=1"), (dim=2|id=458|"Qubit,Site,n=2"), (dim=1|id=909|"Link,l=2"))


((dim=1|id=12|"Link,l=1"), (dim=2|id=458|"Qubit,Site,n=2"), (dim=1|id=909|"Link,l=2"))

In [28]:
@show inds(state0[end])

inds(state0[end]) = ((dim=1|id=531|"Link,l=3"), (dim=2|id=617|"Qubit,Site,n=4"))


((dim=1|id=531|"Link,l=3"), (dim=2|id=617|"Qubit,Site,n=4"))